In [3]:
import os

os.environ['SPARK_HOME'] = '/opt/homebrew/Cellar/apache-spark/4.0.0/libexec'
# os.environ['SPARK_HOME'] = 'spark:///opt/spark'
os.environ['JAVA_HOME'] = '/opt/homebrew/Cellar/openjdk@17/17.0.16/'


In [1]:
# import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, to_date, dayofweek, when, count, make_date, regexp_replace, round, floor
from pyspark.sql.functions import year, month, dayofmonth

# findspark.init()

In [ ]:
spark = SparkSession.builder \
    .appName("VSCode to Docker Spark") \
    .master("spark://localhost:7077") \
    .config("spark.driver.host", "host.docker.internal") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/admin/spark/./bin/spark-submit'

In [21]:
spark

# 1. 데이터 로딩 및 샘플링

In [12]:
import os

FILE_NAME_HEADER = "shared/data/fhv_tripdata_2024_1278/fhvhv_tripdata_2024-"
month = '01'

In [ ]:
# BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# INPUT_DATA_PATH = os.path.join(BASE_DIR, 'data')
# OUTPUT_DATA_PATH = os.path.join(BASE_DIR, 'test_data') if is_test_mode else INPUT_DATA_PATH


In [13]:
path = f"{FILE_NAME_HEADER}{month}.parquet"
df = spark.read.parquet(path)

25/08/08 18:02:43 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/admin/softeer_de_wiki/mission/W4/.venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/softeer_de_wiki/mission/W4/.venv/lib/python3.11/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/.pyenv/versions/3.11.13/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [7]:
df.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [8]:
columns = [
    'request_datetime',
    'trip_miles',
    'trip_time',
    'base_passenger_fare'
]

reduced_df = df.select(*columns)
reduced_df.show(5)

+-------------------+----------+---------+-------------------+
|   request_datetime|trip_miles|trip_time|base_passenger_fare|
+-------------------+----------+---------+-------------------+
|2024-01-01 00:21:47|      2.83|     2251|              45.61|
|2024-01-01 00:10:56|      1.57|      432|              10.05|
|2024-01-01 00:20:04|      1.98|      731|              18.07|
|2024-01-01 00:35:46|      1.99|      930|              17.17|
|2024-01-01 00:48:19|      2.65|      761|              38.67|
+-------------------+----------+---------+-------------------+
only showing top 5 rows


In [9]:
reduced_df.printSchema()

root
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)



In [10]:
filtered_df = reduced_df.filter((col('base_passenger_fare') > 0))

filtered_df = filtered_df.withColumn('request_datetime', to_date(col('request_datetime'), 'yyyy-MM-dd'))

# Select 2024 datas
filtered_df = filtered_df.filter(year(col('request_datetime')) == 2024)

In [11]:
filtered_df.show(5)

+----------------+----------+---------+-------------------+
|request_datetime|trip_miles|trip_time|base_passenger_fare|
+----------------+----------+---------+-------------------+
|      2024-01-01|      2.83|     2251|              45.61|
|      2024-01-01|      1.57|      432|              10.05|
|      2024-01-01|      1.98|      731|              18.07|
|      2024-01-01|      1.99|      930|              17.17|
|      2024-01-01|      2.65|      761|              38.67|
+----------------+----------+---------+-------------------+
only showing top 5 rows


In [19]:
filtered_df.printSchema()

root
 |-- request_datetime: date (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)



In [17]:
# 날씨 데이터 로드
weather_path_header = "shared/data/2024_weather/"
weather_df = spark.read.option("header", True).csv(weather_path_header + "01.csv")

In [ ]:
weather_df = weather_df.withColumn("date", make_date("year", "month", "day"))
weather_df = weather_df.drop("year", "month", "day")

for col_name in ['precipitation1', 'precipitation2', 'precipitation3']:
    weather_df = weather_df.withColumn(
        col_name,
        regexp_replace(col(col_name), 'T', '0')
    )

# 타입 변경
for col_name in ['precipitation1', 'precipitation2', 'precipitation3']:
    weather_df = weather_df.withColumn(
        col_name,
        col(col_name).cast('float')
    )

# precipitation 합산
weather_df = weather_df.withColumn(
    "precipitation",
    round(col("precipitation1") + col("precipitation2") + col("precipitation3"), 2)
)

weather_df = weather_df.drop("precipitation1", "precipitation2", "precipitation3")

In [ ]:
weather_df = weather_df \
    .withColumn("max", round((col("max") - 32) * 5 / 9, 1).alias("max")) \
    .withColumn("min", round((col("min") - 32) * 5 / 9, 1).alias("min")) \

df_with_weather = filtered_df.join(weather_df, on="date", how="left")

In [ ]:

df_with_day = df_with_weather.withColumn("date", to_date(col("request_datetime"))) \
    .withColumn("day_of_week", dayofweek(col("date"))) \
    .withColumn("day_type", when(col("day_of_week").isin(1, 7), "weekend").otherwise("weekday"))

In [ ]:
# 평일/주말 기준 집계
result1 = df_with_day.groupBy("day_type").agg(
    count("*").alias("count"),
    avg("trip_miles").alias("avg_trip_miles"),
    avg("base_passenger_fare").alias("avg_base_passenger_fare")
)

In [ ]:
result1.show(5)

+--------+--------+-----------------+-----------------------+
|day_type|   count|   avg_trip_miles|avg_base_passenger_fare|
+--------+--------+-----------------+-----------------------+
| weekday|13703263| 4.89827594135792|      24.48817147783582|
| weekend| 5953333|4.701568428643141|      22.77146345416406|
+--------+--------+-----------------+-----------------------+



In [ ]:
df_with_day = df_with_day.withColumn("temp_group", floor(col("max") / 5) * 5)

result2 = df_with_day.groupBy("temp_group").agg(
    count("*").alias("count"),
    round(avg("trip_miles"), 2).alias("avg_trip_miles"),
    round(avg("base_passenger_fare"), 2).alias("avg_base_passenger_fare")
).orderBy("temp_group")

In [ ]:
result2.show(5)